In [14]:
!pip install fasttext

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 68.8/68.8 kB 4.2 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
  Using cached pybind11-2.10.4-py3-none-any.whl (222 kB)
  Created wheel for fasttext: filename=fasttext-0.9.2-cp310-cp310-linux_x86_64.whl size=299067 sha256=8f0523b25f5c44c5f66dd86bd687a43ccd5d3a08091fcfe95ac00cdbe669fe90
  Stored in directory: /home/conceptelling/.cache/pip/wheels/fc/22/93/1a3d535655339964fd8936d807ec85da466303d545023d2139
Successfully built fasttext


In [15]:
import fasttext

In [16]:
# 사전 훈련된 FastText 모델 다운로드
!wget https://dl.fbaipublicfiles.com/fasttext/vectors-crawl/cc.en.300.bin.gz
!gunzip cc.en.300.bin.gz

--2023-04-28 03:18:31--  https://dl.fbaipublicfiles.com/fasttext/vectors-crawl/cc.en.300.bin.gz
Resolving dl.fbaipublicfiles.com (dl.fbaipublicfiles.com)... 13.249.85.90, 13.249.85.39, 13.249.85.19, ...
Connecting to dl.fbaipublicfiles.com (dl.fbaipublicfiles.com)|13.249.85.90|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 4503593528 (4.2G) [application/octet-stream]
Saving to: ‘cc.en.300.bin.gz’

cc.en.300.bin.gz    100%[===================>]   4.19G   186MB/s    in 23s     

2023-04-28 03:18:55 (184 MB/s) - ‘cc.en.300.bin.gz’ saved [4503593528/4503593528]



In [17]:
# FastText 모델 로드
fasttext_model = fasttext.load_model("cc.en.300.bin")

In [18]:
def get_title_embedding_fasttext(title):
    words = title.split()
    if len(words) == 0:
        return np.zeros(300)
    embeddings = [fasttext_model.get_word_vector(word) for word in words]
    return np.mean(embeddings, axis=0)

In [19]:
title_embeddings = train_data['Book-Title'].apply(get_title_embedding_fasttext).tolist()

In [20]:
empty_titles_count = 0
for title in train_data['Book-Title']:
    words = title.split()
    if not words:
        empty_titles_count += 1

print(f"빈 문자열이거나 토큰화된 단어가 없는 책 제목의 개수: {empty_titles_count}")

빈 문자열이거나 토큰화된 단어가 없는 책 제목의 개수: 0


In [21]:
for idx, emb in enumerate(title_embeddings):
    if not isinstance(emb, np.ndarray) or emb.shape != (300,):
        print(f"Index: {idx}, Title: {train_data['Book-Title'][idx]}, Embedding: {emb}")

In [22]:
from sklearn.decomposition import PCA

# PCA 객체를 생성
pca = PCA(n_components=30)

# 평균 임베딩 벡터로 구성된 리스트를 NumPy 배열로 변환
title_embeddings_array = np.array(title_embeddings)

# 차원 축소
reduced_title_embeddings = pca.fit_transform(title_embeddings_array)

In [24]:
print("reduced_title_embeddings.shape >>> ",reduced_title_embeddings.shape)
print(reduced_title_embeddings)

reduced_title_embeddings.shape >>>  (871393, 30)
[[-1.84071407e-01  7.39709064e-02  3.11346829e-01 ...  9.42201838e-02
   8.59764591e-03 -7.14057684e-02]
 [-3.53375673e-02 -5.47084808e-02 -1.10605009e-01 ... -4.14647348e-02
  -4.33949783e-04  2.02504173e-02]
 [-6.37408793e-02 -6.56020120e-02 -1.57652587e-01 ... -2.49061342e-02
   3.32578644e-02  1.88359581e-02]
 ...
 [ 3.22970003e-01  4.46004570e-01 -1.57090873e-01 ...  1.29034137e-03
  -1.06627531e-02  1.41104646e-02]
 [ 1.62676945e-02 -4.58159819e-02 -3.09608616e-02 ...  6.44903556e-02
   7.00727745e-04  1.44635784e-02]
 [ 1.75540168e-02 -1.24135606e-01 -8.62156451e-02 ...  4.41524014e-02
  -6.25252873e-02 -2.00788826e-02]]
